In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np

import os
sys.path.insert(0, os.path.abspath(''))

from hydroml.libs.model import BaryonModel
from hydroml.libs.traindataset import TrainDataset

from torch.utils.data import DataLoader

from torch import nn

import torch

In [2]:
batch_size = 64
ngpu = 0
# Epochs set to '1' for testing
epochs = 2000
learning_rate = 1e-4
beta1 = 0.001
n_samples = 20000

In [3]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
net_Baryons = np.fromfile('../datasets/training/NetbaryonDis_OSG3DAuAu200_tune18.6_wBulk_22momdeltaf_wHBT_netBaryon.dat', dtype=np.float32)

net_Protons = np.fromfile('../datasets/training/NetbaryonDis_OSG3DAuAu200_tune18.6_wBulk_22momdeltaf_wHBT_netProton.dat', dtype=np.float32)

dataset = TrainDataset(net_Baryons, net_Protons, 141)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [5]:
model = BaryonModel()

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [7]:
loss_func = nn.MSELoss()

In [ ]:
from hydroml.utils.plot import plot_output

loss_total = []

for epoch in range(epochs):
    for i, data in enumerate(data_loader):
        keys = data[0]
        values = data[1]

        output = model(keys)

        loss = loss_func(output, values)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        if (epoch%10 == 0):
            if loss < 0.1:
                plot_output(dataset.eta, output.detach().numpy()[0], values[0])
            print(loss)

In [9]:
torch.save(model, '../models/test_200.pt')